In [20]:
import os
from pathlib import Path
from dataclasses import dataclass

In [8]:
os.chdir("../")

In [9]:
%pwd

'd:\\text\\text-summarizer-project'

In [10]:
%ls

 Volume in drive D is New Volume
 Volume Serial Number is 2029-FA7D

 Directory of d:\text\text-summarizer-project

23/08/2023  11:16 am    <DIR>          .
22/08/2023  06:34 pm    <DIR>          ..
22/08/2023  07:56 pm    <DIR>          .github
22/08/2023  06:34 pm             3,238 .gitignore
23/08/2023  11:16 am    <DIR>          .idea
22/08/2023  07:56 pm                 0 app.py
23/08/2023  10:40 am    <DIR>          build
22/08/2023  07:56 pm    <DIR>          config
22/08/2023  07:56 pm                 0 DockerFile
23/08/2023  11:41 am    <DIR>          experiment
22/08/2023  06:34 pm             1,091 LICENSE
22/08/2023  09:05 pm    <DIR>          logs
22/08/2023  09:01 pm                74 main.py
23/08/2023  11:44 am                10 params.yaml
22/08/2023  10:56 pm               156 README.md
22/08/2023  08:33 pm               290 requirements.txt
23/08/2023  11:08 am               576 setup.py
23/08/2023  10:48 am    <DIR>          src
22/08/2023  07:55 pm             1,33

In [23]:
from text_summarizer.constants import *

In [14]:
from text_summarizer.utils.common import read_yaml,create_dirs

In [15]:
@dataclass(frozen=True)
class DataExtractionConfig:
    root_dir: Path 
    source: str
    local_data: Path 
    unzip_dir:Path 

In [43]:
class ConfigurationManager:
    def __init__(
        self,
        config_path=CONFIG_FILE_PATH,
        param_path=PARAM_FILE_PATH
    ):
        self.config=read_yaml(config_path)
        self.params=read_yaml(param_path)
        create_dirs([self.config.artifacts_root])

    def get_data_extraction_config(self)->DataExtractionConfig:
        create_dirs([self.config.data_extraction.root_dir])
        data_extraction_config=DataExtractionConfig(
            root_dir=self.config.data_extraction.root_dir,
            source= self.config.data_extraction.source,
            local_data=self.config.data_extraction.local_data,
            unzip_dir=self.config.data_extraction.unzip_dir
        )
        return data_extraction_config
        

In [32]:
import urllib.request as request 
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_file_size

### Components

In [33]:
class DataExtraction:
    def __init__(self,config:DataExtractionConfig):
        self.config=config
    def download_file(self):
        if  not os.path.exists(self.config.local_data):
            filename,headers=request.urlretrieve(
                url=self.config.source,
                filename=self.config.local_data
            )
            logger.info(f"File {filename} downloading with headers {headers}")
        else:
            logger.info(f"File {filename} already exits with size {get_file_size(Path(self.config.local_data))} ")
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data,'r') as ref:
            ref.extractall(unzip_path)

## Pipeline

In [44]:
try:
    config=ConfigurationManager()
    data_extraction_config=config.get_data_extraction_config()
    data_extraction=DataExtraction(config=data_extraction_config)
    data_extraction.download_file()
    data_extraction.extract_zip_file()
except Exception as e:
    raise e
    


[2023-08-23 12:26:22,605: INFO: common Successfully loaded config\config.yaml ,yaml file]
[2023-08-23 12:26:22,607: INFO: common Successfully loaded params.yaml ,yaml file]
[2023-08-23 12:26:52,804: INFO: 3046373883 File artifacts/data_extraction/data.zip downloading with headers Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C43C:4D0D:17F25C:1B5910:64E5B4A2
Accept-Ranges: bytes
Date: Wed, 23 Aug 2023 07:26:27 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1245-QPG
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1692775587.525946,VS0,VE629
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-